In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from espm.conf import log_shift, dicotomy_tol, sigmaL
from espm.utils import create_laplacian_matrix
from espm.estimators import SmoothNMF
from espm.models import ToyModel
from copy import deepcopy
from espm.weights import generate_weights as gw
from espm.datasets.base import generate_spim_sample
from espm.estimators.updates import initialize_algorithms



In [ ]:
laplacian = False
noise = True
force_simplex = True



In [ ]:
from convergence import run_experiment_set
seed = 50
max_iter = 1000
losses, final_losses, Ws, Hs, params, captions, gammas, l_infty, W, H, true_D, true_H, X, Xdot, times = run_experiment_set(laplacian, noise, force_simplex, seed=seed, max_iter=max_iter)
losses = np.array(losses) - l_infty.reshape(-1, 1)
shape_2d = params[0][0]["shape_2d"]
k = H.shape[0]


In [ ]:
plt.figure(figsize=[10, 6])
# plt.figure(figsize=[15, 10])

for loss, caption in zip(losses, captions):
    iterations = np.arange(len(loss))+1
    if len(iterations)>10:
        plt.plot(iterations, loss, ".-", label=caption)
max_y = np.max(losses)
min_y = np.min(losses)
plt.ylim([min_y, max_y])
plt.xlim([1, losses.shape[1]])
plt.yscale("log")
plt.xscale("log")
plt.xlabel("Iterations")
plt.legend()

In [ ]:
# for gamma, caption in zip(gammas, captions):
#     iterations = np.arange(len(gamma))+1

#     plt.plot(iterations, gamma, ".", label=caption)
# plt.yscale("log")
# plt.legend()


In [ ]:
Hmat = H.reshape(k, shape_2d[0], shape_2d[1])
Hmat_true = true_H.reshape(k, shape_2d[0], shape_2d[1])
scale = 4
cmap = plt.cm.viridis
plt.figure(figsize=(scale*k,2*scale))
for i in range(k):
    plt.subplot(2,k,i+1)
    plt.imshow(Hmat[i], cmap=cmap, vmin=0, vmax=1)
    plt.title(f"Estimated H {i}")
    plt.axis('off')
    plt.colorbar()
    plt.subplot(2,k,i+1+k)
    plt.imshow(Hmat_true[i], cmap=cmap, vmin=0, vmax=1)
    plt.title(f"True H {i}")
    plt.axis('off')
    plt.colorbar()

plt.tight_layout()


In [ ]:
## Paper plots

In [ ]:
from pathlib import Path
pathfig = Path('./figures/')
pathfig.mkdir(exist_ok=True)

In [ ]:
naming_hash = {}
naming_hash["log_surrogate - no linesearch"] = "MU update\nno linesearch"
naming_hash["log_surrogate -  linesearch"] = "MU update\nlinesearch"
naming_hash["l2_surrogate - no linesearch"] = "Qudratic update\nno linesearch"
naming_hash["l2_surrogate -  linesearch"] = "Qudratic update\nlinesearch"
naming_hash["projected_gradient - no linesearch"] = "Projected gradient\nno linesearch"
naming_hash["projected_gradient -  linesearch"] = "Projected gradient\nlinesearch"

In [ ]:
for laplacian in [False, True]:
    for noise in [False, True]:
        # load data 
        filename = f"losses_{laplacian}_{noise}_{force_simplex}.npz"
        data = np.load(filename, allow_pickle=True)
        losses = data["losses"]
        l_infty = data["l_infty"]
        params = data["params"]
        captions = [naming_hash[d] for d in data["captions"]]
        true_D = data["true_D"]
        true_H = data["true_H"]
        X = data["X"]
        Xdot = data["Xdot"]
        W = data["W"]
        H = data["H"]
        gammas = data["gammas"]
        shape_2d = params[0][0]["shape_2d"]
        k = H.shape[0]
        times = data["times"]
        losses = np.mean(losses-l_infty.reshape(-1, 1, 1), axis=0)


        plt.figure(figsize=[8, 4])
        for loss, caption in zip(losses, captions):
            iterations = np.arange(len(loss))+1
            if len(iterations)>10:
                plt.plot(iterations, loss, ".-", label=caption)
        max_y = np.max(losses)
        min_y = np.min(losses)
        plt.ylim([min_y, max_y])
        plt.xlim([1, losses.shape[1]])
        plt.yscale("log")
        plt.xscale("log")
        plt.xlabel("Iterations")
        plt.legend(loc="lower left")

        fig = plt.gcf()
        filename = f"convergence_{'' if laplacian else 'no_'}laplacian_{'' if noise else 'no_'}noise.pdf"
        fig.savefig(pathfig / Path(filename), bbox_inches='tight')

In [ ]:
shift = 0
plt.figure(figsize=[6, 3])
for laplacian in [True, False]:
    for noise in [False, True]:
        filename = f"losses_{laplacian}_{noise}_{force_simplex}.npz"
        data = np.load(filename, allow_pickle=True)
        captions = [naming_hash[d] for d in data["captions"]]

        times = data["times"]
        times = np.mean(times, axis=0)
        X_axis = np.arange(len(captions))
  
        plt.bar(X_axis - 0.3 + shift, times, 0.2, label = f"{'Smooth' if laplacian else 'random'} nois{'y' if noise else 'less'} data")
        shift += 0.2

plt.xticks(X_axis, captions)
plt.xticks(rotation=45, ha="right")
plt.xlabel("Algorithm")
plt.ylabel("Seconds")
plt.title("Execution time for 1000 iterations")
plt.legend()
plt.show()

fig = plt.gcf()
filename = f"speed_comparison.pdf"
fig.savefig(pathfig / Path(filename), bbox_inches='tight')